In [1]:
import cv2
import os
import glob
import math
import numpy as np
import pandas as pd

import config
from cleaning_with_coordinate_clustering import VideoReader, Metadata

face_coordinates = pd.read_json(config.FACE_COORDINATES_PATH).T

(2,)
[[0, [565.47998046875, 616.5693359375, 721.7264404296875, 805.61962890625]], [1, [570.3684692382812, 614.9842529296875, 722.6734008789062, 799.764892578125]], [2, [571.939453125, 620.2518920898438, 722.45361328125, 802.4014892578125]], [3, [572.754150390625, 617.57568359375, 724.0797119140625, 800.370849609375]]]
nan

In [ ]:
images = glob.glob('D:/DFDC/train_faces/*.jpg')

In [ ]:
len(images)

In [ ]:
all_images = []

for i, im_path in enumerate(images):
    if i % 10000 == 0:
        print(i)
    image = cv2.imread(im_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    all_images.append(image)
np.savez('D:/DFDC/chunk.npz', all_images)

In [2]:
data = np.load('D:/DFDC/chunk.npz', allow_pickle=True)['arr_0']

In [9]:
def display_images(images, cols = 5, titles=None, figsize=(64, 16)):
    plt.figure(figsize=figsize)
    rows = len(images) // cols + 1
    
    if titles is None:
        titles = ['']*len(images)
    
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(rows, cols, i + 1)
        plt.title(title)
        plt.imshow(img)     
        
        plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
        plt.tick_params(
        axis='y',          # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left=False,      # ticks along the bottom edge are off
        labelleft=False) # labels along the bottom edge are off

In [15]:
len(data[6546:6666])

120